In [16]:
from pyspark.mllib.recommendation import ALS,Rating,MatrixFactorizationModel

In [17]:
rawData=sc.textFile('ml-100k/u.data')

In [18]:
rawData.first()

u'196\t242\t3\t881250949'

In [26]:
ratings=rawData.map(lambda x:x.split('\t')).map(lambda l:Rating(int(l[0]),int(l[1]),float(l[2])))

In [27]:
ratings.first()

Rating(user=196, product=242, rating=3.0)

In [25]:
rank=50
iterations=10

In [29]:
model=ALS.train(ratings,rank,iterations)

In [35]:
model.userFeatures().count()

943

In [36]:
model.productFeatures().count()

1682

In [37]:
model.predict(789,123)

3.043096412523513

In [38]:
userId=789
K=10

In [39]:
topKRecs=model.recommendProducts(userId,K)

In [42]:
topKRecs

[Rating(user=789, product=675, rating=5.995524130973601),
 Rating(user=789, product=185, rating=5.801341836507387),
 Rating(user=789, product=183, rating=5.7633685695900745),
 Rating(user=789, product=219, rating=5.639828802484746),
 Rating(user=789, product=223, rating=5.609731984856742),
 Rating(user=789, product=484, rating=5.47318362446274),
 Rating(user=789, product=11, rating=5.270572409677115),
 Rating(user=789, product=479, rating=5.22118635212532),
 Rating(user=789, product=12, rating=5.208941698906468),
 Rating(user=789, product=64, rating=5.180917762674255)]

In [43]:
movies=sc.textFile("ml-100k/u.item")

In [46]:
movies.first()

u'1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0'

In [69]:
titles=movies.map(lambda line:line.split("|")).map(lambda line:(int(line[0]),line[1])).collectAsMap()

In [72]:
titles[123]

u'Frighteners, The (1996)'

In [74]:
ratings.first()

Rating(user=196, product=242, rating=3.0)

In [128]:
movieForUser=ratings.keyBy(lambda x:x.user).lookup(789)

In [129]:
len(movieForUser)

33

In [130]:
movieForUser=sc.parallelize(movieForUser).sortBy(lambda x:x.rating,ascending=False)#先将list转为RDD

In [131]:
movieForUser.first()

Rating(user=789, product=127, rating=5.0)

In [132]:
movieForUser.map(lambda line:(titles[line.product],line.rating)).take(10)

[(u'Godfather, The (1972)', 5.0),
 (u'Trainspotting (1996)', 5.0),
 (u'Dead Man Walking (1995)', 5.0),
 (u'Star Wars (1977)', 5.0),
 (u'Swingers (1996)', 5.0),
 (u'Leaving Las Vegas (1995)', 5.0),
 (u'Bound (1996)', 5.0),
 (u'Fargo (1996)', 5.0),
 (u'Last Supper, The (1995)', 5.0),
 (u'Private Parts (1997)', 4.0)]

In [136]:
topKRecs=sc.parallelize(topKRecs)

In [137]:
topKRecs.map(lambda line:(titles[line.product],line.rating)).take(10)

[(u'Nosferatu (Nosferatu, eine Symphonie des Grauens) (1922)',
  5.995524130973601),
 (u'Psycho (1960)', 5.801341836507387),
 (u'Alien (1979)', 5.7633685695900745),
 (u'Nightmare on Elm Street, A (1984)', 5.639828802484746),
 (u'Sling Blade (1996)', 5.609731984856742),
 (u'Maltese Falcon, The (1941)', 5.47318362446274),
 (u'Seven (Se7en) (1995)', 5.270572409677115),
 (u'Vertigo (1958)', 5.22118635212532),
 (u'Usual Suspects, The (1995)', 5.208941698906468),
 (u'Shawshank Redemption, The (1994)', 5.180917762674255)]